In [ ]:
import pandas as pd
import boto3
from dotenv import load_dotenv
import os
import io

In [ ]:
# Chargement API KEYS
load_dotenv()

API_KEY_S3 = os.environ["API_KEY_S3"]
API_SECRET_KEY_S3 = os.environ["API_SECRET_KEY_S3"]

bucket_name = "renergies99-lead-bucket"
s3_prefix = "public"

In [3]:
# Session Boto3
session = boto3.Session(
    aws_access_key_id=API_KEY_S3,
    aws_secret_access_key=API_SECRET_KEY_S3,
    region_name="eu-west-3",
)

s3 = session.resource("s3")
bucket = s3.Bucket(bucket_name)

# Dossier S3 avec données
national_prefix = f"{s3_prefix}/prod/unzipped/regional/"

# Lister les objets S3 sous ce préfixe
xlsx_keys = sorted(
    obj.key
    for obj in bucket.objects.filter(Prefix=national_prefix)
    if obj.key.endswith(".xlsx")
)

print(f"{len(xlsx_keys)} fichiers trouvés sur S3")

# 2. Charger et concaténer
dfs = []
for key in xlsx_keys:
    s3_obj = s3.Object(bucket_name, key).get()
    buffer = io.BytesIO(s3_obj["Body"].read())

    df = pd.read_excel(buffer)  # ici seulement xlsx → pas besoin de gérer xls
    dfs.append(df)

df_all = pd.concat(dfs, ignore_index=True)

print(df_all.shape)
df_all.head()

5 fichiers trouvés sur S3
(171023, 67)


,Périmètre,Nature,Date,Heures,Consommation,Thermique,Nucléaire,Eolien,Solaire,Hydraulique,...,TCO Nucléaire (%),TCH Nucléaire (%),TCO Eolien (%),TCH Eolien (%),TCO Solaire (%),TCH Solaire (%),TCO Hydraulique (%),TCH Hydraulique (%),TCO Bioénergies (%),TCH Bioénergies (%)
0,Auvergne-Rhône-Alpes,Données temps réel,2025-01-01,00:00:00,9002.0,134.0,11017.0,118.0,0.0,2609.0,...,122.38,81.19,1.31,15.92,0,0.0,28.98,22.88,0.44,21.16
1,Auvergne-Rhône-Alpes,Données temps réel,2025-01-01,00:15:00,9023.0,134.0,10976.0,130.0,0.0,2473.0,...,121.64,80.88,1.44,17.54,0,0.0,27.41,21.69,0.44,21.16
2,Auvergne-Rhône-Alpes,Données temps réel,2025-01-01,00:30:00,8940.0,133.0,10733.0,137.0,0.0,2454.0,...,120.06,79.09,1.53,18.49,0,0.0,27.45,21.52,0.45,21.16
3,Auvergne-Rhône-Alpes,Données temps réel,2025-01-01,00:45:00,8803.0,125.0,10399.0,140.0,0.0,2345.0,...,118.13,76.63,1.59,18.89,0,0.0,26.64,20.57,0.45,21.16
4,Auvergne-Rhône-Alpes,Données temps réel,2025-01-01,01:00:00,8802.0,126.0,10137.0,151.0,0.0,2319.0,...,115.17,74.70,1.72,20.38,0,0.0,26.35,20.34,0.45,21.16


In [4]:
df_all = df_all[~df_all["Heures"].astype(str).str.endswith(("15:00", "45:00"))]

In [5]:
# Sélection automatique des colonnes TCO... (%) ou TCH... (%) si besoin
cols_pct = [c for c in df_all.columns if "TCO" in c or "TCH" in c]

# Conversion en float
df_all[cols_pct] = df_all[cols_pct].apply(
    lambda col: pd.to_numeric(col, errors="coerce")
)

In [6]:
# Convertir le DataFrame en CSV et résidant en mémoire
csv_buffer = io.StringIO()
df_all.to_csv(csv_buffer, index=False, encoding="utf-8")

# Upload direct sur S3
s3_key = "public/prod/eCO2mix_RTE_Auvergne-Rhone-Alpes.csv"

s3.Object(bucket_name, s3_key).put(Body=csv_buffer.getvalue())

print(f"CSV envoyé sur s3://{bucket_name}/{s3_key}")

CSV envoyé sur s3://renergies99-bucket/public/prod/eCO2mix_RTE_Auvergne-Rhone-Alpes.csv


In [7]:
df_all.shape

(85514, 67)

In [8]:
df_all.head()

,Périmètre,Nature,Date,Heures,Consommation,Thermique,Nucléaire,Eolien,Solaire,Hydraulique,...,TCO Nucléaire (%),TCH Nucléaire (%),TCO Eolien (%),TCH Eolien (%),TCO Solaire (%),TCH Solaire (%),TCO Hydraulique (%),TCH Hydraulique (%),TCO Bioénergies (%),TCH Bioénergies (%)
0,Auvergne-Rhône-Alpes,Données temps réel,2025-01-01,00:00:00,9002.0,134.0,11017.0,118.0,0.0,2609.0,...,122.38,81.19,1.31,15.92,0.0,0.0,28.98,22.88,0.44,21.16
2,Auvergne-Rhône-Alpes,Données temps réel,2025-01-01,00:30:00,8940.0,133.0,10733.0,137.0,0.0,2454.0,...,120.06,79.09,1.53,18.49,0.0,0.0,27.45,21.52,0.45,21.16
4,Auvergne-Rhône-Alpes,Données temps réel,2025-01-01,01:00:00,8802.0,126.0,10137.0,151.0,0.0,2319.0,...,115.17,74.70,1.72,20.38,0.0,0.0,26.35,20.34,0.45,21.16
6,Auvergne-Rhône-Alpes,Données temps réel,2025-01-01,01:30:00,8718.0,125.0,9573.0,154.0,0.0,2511.0,...,109.81,70.55,1.77,20.78,0.0,0.0,28.80,22.02,0.46,21.16
8,Auvergne-Rhône-Alpes,Données temps réel,2025-01-01,02:00:00,8719.0,126.0,9603.0,164.0,0.0,2500.0,...,110.14,70.77,1.88,22.13,0.0,0.0,28.67,21.93,0.46,21.16


In [9]:
print(df_all.info())
print(df_all.describe())
print(df_all.shape)

<class 'pandas.core.frame.DataFrame'>
Index: 85514 entries, 0 to 171022
Data columns (total 67 columns):
 #   Column                                                               Non-Null Count  Dtype         
---  ------                                                               --------------  -----         
 0   Périmètre                                                            85514 non-null  object        
 1   Nature                                                               85510 non-null  object        
 2   Date                                                                 85510 non-null  datetime64[ns]
 3   Heures                                                               85510 non-null  object        
 4   Consommation                                                         85510 non-null  float64       
 5   Thermique                                                            85510 non-null  float64       
 6   Nucléaire                                         